<a href="https://colab.research.google.com/github/edisonc57/PE-with-Python/blob/main/PE_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Material Balance Equation

*For a single-phase:*

$$
N= N_o - \Delta N + G - \phi B \Delta p
$$

In [ ]:
def remaining_reserves(N, No, delta_N, G, phi, B, delta_P):
    remaining_volume = No - delta_N + G - phi * B * delta_P
    return remaining_volume

N = 1000000        # Reservas iniciales en barriles
No = 1200000       # Reservas originales estimadas en barriles
delta_N = 200000   # Reducción debido a la extracción en barriles
G = 50000          # Aumento de reservas debido a nuevos descubrimientos en barriles
phi = 0.1          # Factor de reducción por pérdida o ineficiencia
B = 10000          # Tasa de producción en barriles por día
delta_P = 0.05     # Cambio en la presión del yacimiento

remaining_reserves(N, No, delta_N, G, phi, B, delta_P)



1049950.0

*For a two-phase:*

$$
N= N_o - \Delta N + E - D  - \phi (B_o \Delta P_o + B_g \Delta P_g )
$$

In [ ]:
def two_phases_remaining_reserves(N, No, delta_N, E, D, phi, Bo, Bg,  delta_Po, delta_Pg):
    remaining_volume = No - delta_N + E - D - phi * (Bo * delta_Po + Bg * delta_Pg)
    return remaining_volume

N = 1000       # Número total de reservas estimadas inicialmente
No = 500       # Volumen inicial de reservas antes de la extracción
delta_N = 50   # Cambio en el volumen de reservas debido a la producción de petróleo
E = 200        # Volumen de inyección de fluidos (agua, gas, etc.) en el reservorio
D = 150        # Volumen de fluidos extraídos del reservorio
phi = 0.1      # Porosidad efectiva del reservorio
Bo = 1.2       # Factor volumétrico de formación del petróleo
Bg = 0.8       # Factor volumétrico de formación del gas
delta_Po = 0.05 # Cambio en la presión del reservorio de petróleo
delta_Pg = 0.03 # Cambio en la presión del reservorio de gas

two_phases_remaining_reserves (N, No, delta_N, E, D, phi, Bo, Bg,  delta_Po, delta_Pg)

499.9916

# Darcy´s Law for fluid flow

Linear flow:
$$
q= -k \frac{A \Delta P}{\mu L}
$$

In [ ]:
def linear_flow(k, A, delta_P, mu, L):
    q = -k * A * delta_P / (mu * L)
    return q

# Ejemplo de valores
k = 1e-12        # Permeabilidad en m²
A = 0.01         # Área en m²
delta_P = 10000  # Diferencia de presión en Pa
mu = 0.001       # Viscosidad en Pa·s
L = 1.0          # Longitud en m

linear_flow(k, A, delta_P, mu, L)

-1e-07

Radial flow:

$$
q= -2 \pi kh \frac{\Delta P} {\mu} (\frac{r_1 r_2} {ln(\frac {r_2}{r_1})})
$$

In [ ]:
import math
def radial_flow(k, h, delta_P, mu, r1, r2):
    q = -2 * math.pi * k * h * delta_P * r1 * r2  / (mu * math.log(r2/r1))
    return q

k = 1e-12        # Permeabilidad en m²
h = 50           # Espesor en m
delta_P = 5000   # Diferencia de presión en Pa
mu = 0.001       # Viscosidad en Pa·s
r1 = 0.1         # Radio del pozo en m
r2 = 100         # Radio de influencia en m

radial_flow(k, h, delta_P, mu, r1, r2)

-0.0022739605897364027

#Volumetric Calculations

$$
N_o = N_i + \frac {A h (1 - S_w)} {\phi B_o}. (\Delta P)
$$

In [ ]:
def calculate_original_in_place(Ni, A, h, Sw, delta_p, phi, Bo):
    No = (A * h * (1 - Sw) * delta_p) / (phi * Bo)
    return No

# Ejemplo de valores
Ni= 100000  # Cantidad de petróleo inicial en bbl
A = 640       # Área del yacimiento en acres
h = 50        # Espesor del yacimiento en pies
S_w = 0.2     # Saturación de agua (20%)
phi = 0.25    # Porosidad del yacimiento (25%)
Bo = 1.2     # Factor de volumen de formación del petróleo
delta_P = 100 # Diferencia de presión en psi

calculate_original_in_place(Ni, A, h, S_w, delta_P, phi, Bo)

8533333.333333334

# PVT Calculations



*Gas compressibility factor Z*

$$
P_{pr}= \frac {P} {P_c}
$$

$$
T_{pr}= \frac {T + 460} {T_c}
$$

In [ ]:
def calculate_gas_compressibility_factor(P, T, P_c, T_c):
  P_pr = P/P_c
  T_pr = T/T_c
  Z= interpolate_z_from_chart(P_pr, T_pr)

def interpolate_z_from_chart(P_pr, T_pr):
    return Z

*Empirical Correlations: Hall-Yarborough*

$$
Z= A + \frac {B} {P_r} + \frac {C} {P_r^2} +\frac {D} {P_r^3} +\frac {E} {P_r^4}
$$

In [2]:
def calculate_Z_factor_hall_yarborough(Pr, Tr):
  A = 0.06125/Tr * math.exp(-1.2*(1-Tr)**2)
  B = 0.0658/Tr * math.exp(-3.5*(1-Tr)**2)
  C = 3.537 *Pr/Tr * math.exp(-7.6 * (1-Tr)**2)
  D = 0.027 * Pr/Tr * math.exp(-1.2 * (1-Tr)**2)
  E = -1.45 * Pr/Tr * math.exp(-3* (1-Tr)**2)

  Z= A + B/P_pr + C/P_pr**2 + D/P_pr**3 + E/P_pr**4

  return Z

*Empirical Correlations: Hall-Yarborough*

$$
Z= Z_1 + (Z_2 - Z_1 -d(Z_3 - Z_2)) e^{- \omega \alpha^2}
$$

$$
\alpha = \frac {\kappa} {P_r}
$$

In [4]:
def calculate_Z_factor_DAK(Pr, Tr, omega):
  Z1 = 0.3265 - 0.0701 /Tr- 0.011 / omega
  Z2 = 0.27 * Pr / Tr
  Z3 = 0.5475 - 0.7361 / Tr + 0.1844 / Tr **2
  d =0.27 * Tr / Tr
  kappa = 0.37464 + 1.54226 * omega - 0.26992 * omega **2

  alpha= kappa/Pr
  exponent_term = math.exp(omega * alpha**2)
  Z= Z1 + (Z2 - Z1 -d*(Z3 - Z2)) * math.exp(-exponent_term)

  return Z
